In [3]:
import numpy as np

from PIL import Image, ImageDraw
import torch
import torchvision
from IPython.display import display

np.random.seed(42)

In [4]:
train_Images_folder = '/Users/abdulrehman/Desktop/SML Project/Wider/WIDER_train/images/'
test_Images_folder = '/Users/abdulrehman/Desktop/SML Project/Wider/WIDER_val/images/'
train_bbx = '/Users/abdulrehman/Desktop/SML Project/Wider/wider_face_split/wider_face_train_bbx_gt.txt'
test_bbx = '/Users/abdulrehman/Desktop/SML Project/Wider/wider_face_split/wider_face_val_bbx_gt.txt'
data_folder = '/Users/abdulrehman/Desktop/SML Project/Faces'

Need to remove occlusion
need to remove invalid images

In [5]:
def create_data_dict(bbx_path,images_path):
    data = {}
    with open(bbx_path, 'r') as file:
        while True:
            line = file.readline()[:-1]#reads the image path
            if not line: break
            faces = []
            n = int(file.readline())#reads the number of bounding boxes present in the image at Path(line)
            check_valid = 1
            im = Image.open(images_path+line)
            l,w,_ = np.array(im).shape
            if n>0:
                while n>0:
                    #reads bounding boxes values and types
                    v = np.fromstring(file.readline(), dtype = float, sep=' ')
                    if check_valid ==1:
                    #if not clear or not valid or contains heavy occlusion do not consider
                        if v[4]!=0 or v[7]==1 or v[8]==2:
                            check_valid = 0
                        area = v[2]*v[3]
                        if area<100:
                            check_valid = 0
                        faces.append([v[0]/w, v[1]/l, v[2]/w, v[3]/l])
                    n-=1
                if check_valid:
                    data[line]=np.array(faces)
#                     print(data[line])
            else: line = file.readline()
    
    return data

In [6]:
train_data = create_data_dict(train_bbx,train_Images_folder)
train_data
# test_data = create_data_dict(test_bbx,test_Images_folder)

{'0--Parade/0_Parade_marchingband_1_849.jpg': array([[0.43847656, 0.23826715, 0.11914062, 0.10758123]]),
 '0--Parade/0_Parade_Parade_0_904.jpg': array([[0.35253906, 0.06843575, 0.25683594, 0.23673184]]),
 '0--Parade/0_Parade_marchingband_1_483.jpg': array([[0.56933594, 0.49348958, 0.05566406, 0.08072917],
        [0.66015625, 0.38932292, 0.04394531, 0.078125  ],
        [0.6015625 , 0.35807292, 0.03808594, 0.05338542],
        [0.60253906, 0.29166667, 0.03613281, 0.04817708],
        [0.44238281, 0.3828125 , 0.04296875, 0.06901042],
        [0.35253906, 0.36588542, 0.03222656, 0.05598958],
        [0.36523438, 0.49479167, 0.04882812, 0.07161458],
        [0.24414062, 0.5390625 , 0.04101562, 0.06901042],
        [0.47363281, 0.54557292, 0.046875  , 0.08463542]]),
 '0--Parade/0_Parade_Parade_0_577.jpg': array([[0.28710938, 0.18951358, 0.44433594, 0.36765635]]),
 '0--Parade/0_Parade_Parade_0_883.jpg': array([[0.51953125, 0.16      , 0.21386719, 0.27333333]]),
 '0--Parade/0_Parade_marching

In [7]:
from shutil import copyfile
from pathlib import Path

def create_dataset(data,dataset_type):
    if dataset_type == 'train':
        Images_folder = train_Images_folder
    else:
        Images_folder = test_Images_folder
    
    images_path = Path(data_folder+'/images/'+dataset_type)
    images_path.mkdir(parents=True, exist_ok=True)
    labels_path = Path(data_folder+'/labels/'+dataset_type)
    labels_path.mkdir(parents=True, exist_ok=True)
    
    name = 1
    for path in data:
        copyfile(Images_folder+path, images_path/(str(name)+'.jpg'))
        with open(labels_path/(str(name)+'.txt'),'w') as out_file:
            for bbx in data[path]:
                x,y,w,h = [float(i) for i in bbx]
                out_file.write('0 '+str(x+w/2)+' '+str(y+h/2)+' '+str(w)+' '+str(h)+'\n')
        name+=1

In [8]:
create_dataset(train_data,'train')

In [9]:
test_data = create_data_dict(test_bbx,test_Images_folder)
test_data

{'0--Parade/0_Parade_Parade_0_194.jpg': array([[0.10839844, 0.30619597, 0.11914062, 0.09149856],
        [0.20410156, 0.25      , 0.06835938, 0.07420749],
        [0.359375  , 0.1815562 , 0.08691406, 0.09582133],
        [0.54199219, 0.20317003, 0.08691406, 0.07204611],
        [0.69042969, 0.1815562 , 0.08984375, 0.09582133]]),
 '0--Parade/0_Parade_Parade_0_829.jpg': array([[0.48925781, 0.11721612, 0.27832031, 0.32454212]]),
 '0--Parade/0_Parade_Parade_0_960.jpg': array([[0.046875  , 0.25830258, 0.109375  , 0.19434194],
        [0.2265625 , 0.0295203 , 0.06640625, 0.12300123],
        [0.6953125 , 0.29520295, 0.06445312, 0.12300123],
        [0.85742188, 0.23616236, 0.078125  , 0.13284133]]),
 '0--Parade/0_Parade_Parade_0_901.jpg': array([[0.00683594, 0.34131737, 0.19628906, 0.29255774],
        [0.27636719, 0.58169376, 0.16015625, 0.17792985],
        [0.6953125 , 0.30025663, 0.28515625, 0.31993157]]),
 '0--Parade/0_Parade_marchingband_1_353.jpg': array([[0.25683594, 0.27789934, 0.11

In [10]:
create_dataset(test_data,'test')

In [11]:
def draw_bounding_boxes(im, bbxs,color='red',width=3):
    draw = ImageDraw.Draw(im)
    l,w,_ = [float(i) for i in np.array(im).shape]
    for box in bbxs:
        x0 = (box[1]-box[3]/2)*w
        y0 = (box[2]-box[4]/2)*l
        x1 = (box[1]+box[3]/2)*w
        y1 = (box[2]+box[4]/2)*l
        draw.rectangle([x0,y0,x1,y1], fill=None, outline=color, width=width)
    return(im)

Retrieving Random images to check of dataset was created correctly.

In [18]:
image_name = '100'
bbxs = []
with open(data_folder+'/labels/test/'+image_name+'.txt', 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        line = np.fromstring(line, dtype=float, sep=' ')
        
        bbxs.append(line)

im = Image.open(data_folder+'/images/test/'+image_name+'.jpg')
im = draw_bounding_boxes(im,bbxs)
np.array(im).shape
# im.show()

(684, 1024, 3)